## Введение в анализ данных, ИАД-2

## НИУ ВШЭ, 2018-19 учебный год

### Домашнее задание №3

Задание выполнил(а): _(впишите свои фамилию и имя)_

### Общая информация

__Дата выдачи:__ 06.05.2019

__Дедлайн:__ 23:59 19.05.2019

### Оценивание и штрафы

Максимальное число баллов, которое можно получить за обязательную часть — 10, а максимальное число дополнительных баллов, которые пойдут в бонус — 0.5.

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи

Загрузка файлов с решениями происходит в системе [Anytask](https://anytask.org/).

Инвайт для группы ИАД-2: zCH4F32

In [270]:
import numpy as np
from sklearn.datasets import load_breast_cancer, load_boston
from sklearn.metrics import accuracy_score, mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.base import BaseEstimator
from scipy import optimize
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import tqdm

# Random Forest своими руками (4)

Реализуйте алгоритм случайного леса. В соответствии с построением алгоритма, ваша реализация должна сочетать в себе метод бэггинга и метод случайных подмножеств. Также она должна работать для двух типов задач: как для задачи регрессии, так и для задачи классификации.

*Примечание. Вы можете использовать `DecisionTree` из `sklearn`.*

In [36]:
class RandomForest:
    def __init__(self, n_estimators, max_depth, max_features='sqrt', task_type='regression'):
        """
        PARAMETERS:
        n_estimators - number of trees in the ensemble
        max_depth - maximum depth of a tree
        max_features - maximum features for each split in DecisionTree from sklearn
        task_type - 'regression' or 'classification'
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.task_type = task_type
        self.trees = []
        
    def fit(self, X_train, y_train):
        """
        INPUT:
        X_train - np.array of shape (k, d)
        y_train - np.array of shape (k,)
        """
        self.trees = []


        for tree in range(self.n_estimators):
            bootstrap = np.array([])
            y_bootstrap = np.array([])
            
            bootstrap_objects = np.random.randint(0, X_train.shape[0], size = X_train.shape[0])
            bootstrap = X_train[bootstrap_objects]
            y_bootstrap = y_train[bootstrap_objects]
            

            if self.task_type == 'regression':
                tree = DecisionTreeRegressor(max_features=self.max_features, max_depth=self.max_depth)
            elif self.task_type == 'classifier':
                tree = DecisionTreeClassifier(max_features=self.max_features, max_depth=self.max_depth)
            tree.fit(bootstrap, y_bootstrap)
            self.trees.append(tree)
            
        return self
        
    def predict(self, X_test):
        """
        INPUT:
        X_test - np.array of shape (m, d)
        
        OUTPUT:
        y_pred - np.array of shape (m,)
        """
        if self.task_type == 'regression':
            return np.mean([tree.predict(X_test) for tree in self.trees], axis = 0)
        elif self.task_type == 'classifier':
            
            prediction = np.mean([tree.predict(X_test) for tree in self.trees], axis = 0)
            for tree in prediction:
                itemindex = np.where(prediction==tree)
                """
                Да, в теории мы подсчитываем количество 'голосов' за каждый класс. Но я считаю, что проще оценить среднее значение. 
                По сути, эти задачи эквивалентны.
                """
                if tree > 0.5:
                    prediction[itemindex[0][0]] = 1
                else:
                    prediction[itemindex[0][0]] = 0
            
            return prediction.astype(int)
    

Проверьте вашу реализацию на задаче регрессии. Подберите оптимальные гиперпараметры, чтобы получить MSE < 10.25 **(не меняйте параметры сида)**.

In [5]:
boston = load_boston()
X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [482]:
np.random.seed(19052019)
mse = 1000000.0
n_estimators = 0
max_depth = 0
for tree in range(30, 120):
    for depth in range(10, 50):
        rf = RandomForest(tree, depth)
        rf.fit(X_train, y_train)
        if mean_squared_error(y_test, rf.predict(X_test)) < mse:
            mse = mean_squared_error(y_test, rf.predict(X_test))
            n_estimators = tree
            max_depth = depth
            
print(f"MSE: {mse} n_estimators: {n_estimators} | max_depth: {max_depth}")

MSE: 8.364135448812341 n_estimators: 49 | max_depth: 31


Проверьте вашу реализацию на задаче классификации. Подберите оптимальные гиперпараметры, чтобы получить accuracy > 0.83 **(не меняйте параметры сида)**.

In [4]:
breast_cancer = load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [541]:
np.random.seed(19052019)

accuracy = 0
n_estimators = 0
max_depth = 0

for tree in range(30, 120):
    for depth in range(10, 50):
        rf = RandomForest(tree, depth, 'sqrt', 'classifier')
        rf.fit(X_train, y_train)
        if accuracy_score(y_test, rf.predict(X_test)) > accuracy:
            accuracy = accuracy_score(y_test, rf.predict(X_test))
            n_estimators = tree
            max_depth = depth
            
print(f"accuracy: {accuracy} n_estimators: {n_estimators} | max_depth: {max_depth}")


accuracy: 0.9790209790209791 n_estimators: 38 | max_depth: 23


# Градиентный бустинг своими руками (4)

Реализуйте алгоритм градиентного бустинга для регрессии. Напомним основные формулы.

Обозначим текущую композицию на $N-1$ шаге за $a_{N - 1}(x_i)$. Следующий базовый алгоритм $b_N(x_i)$ обучается на ответах $-\frac{\partial L(y_i, z)}{\partial z}\Bigl|_{z = a_{N - 1}(x_i)}$, где $L(y_i, z)$ — значение функции потерь на объекте при правильном ответе $y_i$ и предсказании $z$. Композиция на следующем шаге получается следующим образом:

$$
a_N(x_i) = a_{N-1}(x_i) + \nu\gamma_Nb_N(x_i)
$$

Здесь $\nu$ — гиперпараметр learning rate, $\gamma_N$ — оптимальный вес, настраиваемый на каждом шаге алгоритма, который можно найти по следующей формуле (обратите внимание на отсутствие $\nu$):

$$
\gamma_N = \mathrm{arg}\min_\gamma \frac{1}{\ell}\sum\limits_{i=1}^{\ell}L\left(y_i, a_{N - 1}(x_i) + \gamma b_N(x_i)\right)
$$

Можете принять $\gamma_N = 1$ для каждого $N$. Однако вы можете получить дополнительные 0.5 баллов за реализацию нахождения оптимального $\gamma_N$ на каждом шаге.

В качестве функции потерь возьмите MSE.

*Примечание. Вы можете использовать `DecisionTree` из `sklearn` и методы оптимизации из различных библиотек.*

In [558]:

class GradientBoosting(BaseEstimator):
    def __init__(self, n_estimators=5, max_depth=5, learning_rate=0.1):
        """
        PARAMETERS:
        n_estimators - number of trees in the ensemble
        max_depth - maximum depth of a tree
        learning_rate - coefficient by which new algorithm result is multiplied
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate 
        self.check = lambda y: np.mean(y) * np.ones([y.shape[0], 1])
        self.trees = []
        self.loss = []
        
    def f(self, x):
        return np.sum((self.y_train - self.answers - x * self.bn) ** 2)

    
    def gradient_mse(self, y, answers):
        return 2 * (answers - y) #/ y.shape[0]
    
        
    def fit(self, X_train, y_train):
        """
        INPUT:
        X_train - np.array of shape (k, d)
        y_train - np.array of shape (k,)
        """
        self.w_values = []
        self.trees = []
        self.y_train = y_train
        self.constant_b = np.repeat(np.mean(y_train, axis = 0), X_train.shape[0]) 
        self.answers = self.constant_b.copy()
        for i in range(self.n_estimators):
            #if i == 0:
            #    r = y_train
            #else:
            #    r = - self.gradient_mse(y_train, self.answers)
            r = - self.gradient_mse(y_train, self.answers)
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X_train, r)
            self.bn = tree.predict(X_train)
            self.trees.append(tree)

            

            w = optimize.differential_evolution(self.f, ((0, 3),) ).x[0]
            self.w_values.append(w)

              
            self.answers += self.learning_rate * self.bn * w

            #self.loss.append(mean_squared_error(y_train, answers))
            
        self.train_pred = self.answers
        return self
        
    def predict(self, X_test):
        """
        INPUT:
        X_test - np.array of shape (m, d)
        
        OUTPUT:
        y_pred - np.array of shape (m,)
        """
        y_pred = np.repeat(np.mean(self.y_train, axis = 0), X_test.shape[0]) 
        i = 0
        for tree in self.trees:

            y_pred += self.learning_rate * tree.predict(X_test) * self.w_values[i]
            i += 1

        return y_pred

Проверьте вашу реализацию. Подберите оптимальные гиперпараметры, чтобы получить MSE < 10 как в обычном случае, так и при нахождении оптимального шага **(не меняйте параметры сида)**.

In [375]:
X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [585]:
np.random.seed(19052019)
gb = GradientBoosting(25, 6, learning_rate=0.2)
gb.fit(X_train, y_train)
mean_squared_error(y_test, gb.predict(X_test))

9.846536950181664

Для удобства проверяющего укажите, пожалуйста, реализовали ли вы нахождение оптимального шага?

# Да

# Крестики-нолики (2)

In [586]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

В этом задании вы поработаете с [данными](https://datahub.io/machine-learning/tic-tac-toe-endgame), состоящими из конечных позиций в игру крестики-нолики. Признаками являются клетки, их значениями — то, что изображено в данной клетке на момент окончания игры (крестик, нолик или пусто). Стоит задача бинарной классификации — определить, выиграли крестики (класс `True`) или нолики (класс `False`). Напомним, что одна из сторон выигрывает в том случае, если либо одна из диагоналей, либо одна из горизонталей, либо одна из вертикалей заполнена соответствующими символами.

In [749]:
data = pd.read_csv('tic-tac-toe.csv')
X = data.drop('class', axis=1)
y = data['class'].astype(int)
data.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [750]:
X = X.applymap(lambda y: 1 if y == 'x' else -1 if y == 'o' else 0)
X.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,1,1,1,1,-1,-1,1,-1,-1
1,1,1,1,1,-1,-1,-1,1,-1
2,1,1,1,1,-1,-1,-1,-1,1
3,1,1,1,1,-1,-1,-1,0,0
4,1,1,1,1,-1,-1,0,-1,0


Применим метод случайного леса к данным и посмотрим на среднее значение accuracy на кросс-валидации.

In [733]:
rf = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=19052019)
score = cross_val_score(rf, X, y, cv=5, scoring='accuracy').mean()
print(score)

0.8917722942607311


Задача: улучшите данное значение до >0.945 с помощью Feature Engineering.

In [751]:
X['Top win'] = X['TL'] + X['TM'] + X['TR']
X['Medium win'] = X['ML'] + X['MM'] + X['MR']
X['Bottom win'] = X['BL'] + X['BM'] + X['BR']
X['Diagonl L win'] = X['TL'] + X['MM'] + X['BR']
X['Diagonl R win'] = X['BL'] + X['MM'] + X['TR']

In [752]:
X.loc[X['Top win'] == 1, 'Top win'] = 0
X.loc[X['Top win'] == 2, 'Top win'] = 0
X.loc[X['Top win'] == -1, 'Top win'] = 0
X.loc[X['Top win'] == -2, 'Top win'] = 0

In [753]:
X.loc[X['Medium win'] == 1, 'Medium win'] = 0
X.loc[X['Medium win'] == 2, 'Medium win'] = 0
X.loc[X['Medium win'] == -1, 'Medium win'] = 0
X.loc[X['Medium win'] == -2, 'Medium win'] = 0

In [754]:
X.loc[X['Bottom win'] == 1, 'Bottom win'] = 0
X.loc[X['Bottom win'] == 2, 'Bottom win'] = 0
X.loc[X['Bottom win'] == -1,'Bottom win'] = 0
X.loc[X['Bottom win'] == -2,'Bottom win'] = 0

In [755]:
X.loc[X['Diagonl L win'] == 1, 'Diagonl L win'] = 0
X.loc[X['Diagonl L win'] == 2, 'Diagonl L win'] = 0
X.loc[X['Diagonl L win'] == -1,'Diagonl L win'] = 0
X.loc[X['Diagonl L win'] == -2,'Diagonl L win'] = 0

In [756]:
X.loc[X['Diagonl R win'] == 1, 'Diagonl R win'] = 0
X.loc[X['Diagonl R win'] == 2, 'Diagonl R win'] = 0
X.loc[X['Diagonl R win'] == -1,'Diagonl R win'] = 0
X.loc[X['Diagonl R win'] == -2,'Diagonl R win'] = 0

In [758]:
X['win'] = X['Top win'] + X['Medium win'] + X['Bottom win'] + X['Diagonl L win'] + X['Diagonl R win']

In [760]:
X.drop(columns=['Top win', 'Medium win', 'Bottom win', 'Diagonl L win', 'Diagonl R win'])

,TL,TM,TR,ML,MM,MR,BL,BM,BR,win
0,1,1,1,1,-1,-1,1,-1,-1,3
1,1,1,1,1,-1,-1,-1,1,-1,3
2,1,1,1,1,-1,-1,-1,-1,1,3
3,1,1,1,1,-1,-1,-1,0,0,3
4,1,1,1,1,-1,-1,0,-1,0,3
5,1,1,1,1,-1,-1,0,0,-1,3
6,1,1,1,1,-1,0,-1,-1,0,3
7,1,1,1,1,-1,0,-1,0,-1,3
8,1,1,1,1,-1,0,0,-1,-1,3
9,1,1,1,1,0,-1,-1,-1,0,3


In [761]:
rf = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=19052019)
score = cross_val_score(rf, X, y, cv=5, scoring='accuracy').mean()
print(score)

0.9685972384233514


Опишите свои рассуждения в 2-3 предложениях. Почему выбранный вами метод сработал?